## Delete resources on KVM

When you are finished with your resources, delete them to free them for other experimenters.

### Configure environment

In [ ]:
import openstack, chi, chi.ssh, chi.network, chi.server, os

In this section, we configure the Chameleon Python client.

For this experiment, we’re going to use the KVM@TACC site, which we indicate below.

We also need to specify the name of the Chameleon “project” that this experiment is part of. The project name will have the format “CHI-XXXXXX”, where the last part is a 6-digit number, and you can find it on your [user dashboard](https://chameleoncloud.org/user/dashboard/).

In the cell below, replace the project ID with your own project ID, then run the cell.

In [ ]:
chi.use_site("KVM@TACC")
PROJECT_NAME = "CHI-XXXXXX"
chi.set("project_name", PROJECT_NAME)

# configure openstacksdk for actions unsupported by python-chi
os_conn = chi.clients.connection()


### Define configuration for this experiment (3 VMs)

For this specific experiment, we will need three virtual machines connected to a common network. Each of the virtual machines will be of the `m1.large` type, with 4 VCPUs, 8 GB memory, 40 GB disk space.

In [ ]:
username = os.getenv('USER')

node_conf = [
 {'name': "node-0",  'flavor': 'm1.large', 'image': 'CC-Ubuntu20.04', 'packages': ["virtualenv"], 'bastion': True}, 
 {'name': "node-1",  'flavor': 'm1.large', 'image': 'CC-Ubuntu20.04', 'packages': [], 'bastion': False}, 
 {'name': "node-2",  'flavor': 'm1.large', 'image': 'CC-Ubuntu20.04', 'packages': [], 'bastion': False} 
]
net_conf = [
 {"name": "net0", "subnet": "192.168.1.0/24", "nodes": [{"name": "node-0",   "addr": "192.168.1.10"}, {"name": "node-1", "addr": "192.168.1.11"}, {"name": "node-2", "addr": "192.168.1.12"}]},
]
route_conf = []

### Delete resources

When you finish your experiment, you should delete your resources! The following cells deletes all the resources in this experiment, freeing them for other experimenters.

In [ ]:
# delete the nodes
server_ids = [chi.server.get_server_id(n['name'] + "_" + username) for n in node_conf]
server_ips = [d['addr'] for s in server_ids for d in chi.server.show_server(s).addresses['public_net_' + username] if d['OS-EXT-IPS:type']=='floating']
for server_id in server_ids:
    chi.server.delete_server(server_id)

In [ ]:
# release the floating IP addresses used for SSH
for server_ip in server_ips:
    ip_details = chi.network.get_floating_ip(server_ip)
    chi.neutron().delete_floatingip(ip_details["id"])

In [ ]:
# delete the router used for public Internet access
router = chi.network.get_router("inet_router_" + username)
public_subnet = chi.network.get_subnet("public_subnet_" + username)
public_net = chi.network.get_network("public_net_" + username)
chi.network.remove_subnet_from_router(router.get("id"), public_subnet.get("id"))
chi.network.delete_router(router.get("id"))

In [ ]:
# delete the public network
chi.network.delete_subnet(public_subnet.get('id'))
chi.network.delete_network(public_net.get("id"))

In [ ]:
# delete the experiment networks
subnets = [chi.network.get_subnet("exp_subnet_" + n['name']  + '_' + username) for n in net_conf]
nets    = [chi.network.get_network("exp_" + n['name']  + '_' + username) for n in net_conf]
for subnet, net in zip(subnets, nets):
    chi.network.delete_subnet(subnet.get('id'))
    chi.network.delete_network(net.get('id'))